In [1]:
import numpy as np
import simple_dmrg.dmrg_functions as dmrg
import simple_dmrg.mps_functions as mps_func
import simple_dmrg.mpo_operations as mpo_ops
import simple_dmrg.mpo_construction as make_mpo
import scipy.sparse as sps
import time
from pathlib import Path

In [2]:
tensor = np.array([[0, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 0]])

In [3]:
eiv, eivc = np.linalg.eig(tensor)

In [4]:
print(eiv)

[ 1. -1.  0.  0.]


In [5]:
# phys dim 4, each site has 4 states: |0>, |u>, |d>, |2>
# |0> is the vacuum state
# u for spin up, d for spin down, |2> := a_u_dag.a_d_dag|0> for both up and down
a_u_2darray = np.array([[0, 1, 0, 0],
                        [0, 0, 0, 0],
                        [0, 0, 0, 1],
                        [0, 0, 0, 0]])
a_d_2darray = np.array([[0, 0, 1, 0],
                        [0, 0, 0, -1],
                        [0, 0, 0, 0],
                        [0, 0, 0, 0]])
a_u = sps.csr_matrix(a_u_2darray)
a_d = sps.csr_matrix(a_d_2darray)
a_u_dag = a_u.transpose().conj()
a_d_dag = a_d.transpose().conj()
n_u = a_u_dag @ a_u
n_d = a_d_dag @ a_d
n_part = n_u + n_d

print(f"a_u = \n{a_u.toarray()}")
print(f"a_d = \n{a_d.toarray()}")
print(f"a_u_dag = \n{a_u_dag.toarray()}")
print(f"a_d_dag = \n{a_d_dag.toarray()}")
print(f"n_u = \n{n_u.toarray()}")
print(f"n_d = \n{n_d.toarray()}")
print(f"n_part = \n{n_part.toarray()}")

a_u = 
[[0 1 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]]
a_d = 
[[ 0  0  1  0]
 [ 0  0  0 -1]
 [ 0  0  0  0]
 [ 0  0  0  0]]
a_u_dag = 
[[0 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 1 0]]
a_d_dag = 
[[ 0  0  0  0]
 [ 0  0  0  0]
 [ 1  0  0  0]
 [ 0 -1  0  0]]
n_u = 
[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 1]]
n_d = 
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]
n_part = 
[[0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 2]]


In [6]:
def build_fermionic_operators(num_sites):
    """
    Build creation and annihilation operators corresponding to each spin site 
    Args:
        num_sites: Number of spatial sites

    Returns: 4 lists of creation and annihilation operators

    """
    a_u_list = []
    a_d_list = []
    a_u_dag_list = []
    a_d_dag_list = []
    id = sps.eye(4, format='csr')
    parity_2darray = np.array([[1, 0, 0, 0],
                               [0, -1, 0, 0],
                               [0, 0, -1, 0],
                               [0, 0, 0, 1]])
    parity = sps.csr_matrix(parity_2darray)
    for i in range(num_sites):
        for j in range(num_sites):
            if j == 0:
                if i == 0:
                    a_u_local = a_u
                    a_d_local = a_d
                else:
                    a_u_local = parity
                    a_d_local = parity
            elif j < i:
                a_u_local = sps.kron(a_u_local, parity)
                a_d_local = sps.kron(a_d_local, parity)
            elif j == i:
                a_u_local = sps.kron(a_u_local, a_u)
                a_d_local = sps.kron(a_d_local, a_d)
            else:
                a_u_local = sps.kron(a_u_local, id)
                a_d_local = sps.kron(a_d_local, id)
        a_u_list.append(a_u_local)
        a_d_list.append(a_d_local)
        a_u_dag_list.append(a_u_local.transpose())
        a_d_dag_list.append(a_d_local.transpose())
    

    return a_u_list, a_d_list, a_u_dag_list, a_d_dag_list

def build_number_operators(a_u_list, a_d_list, a_u_dag_list, a_d_dag_list):
    num_sites = len(a_u_list)
    n_u_list = []
    n_d_list = []
    n_part_list = []

    for i in range(num_sites):
        n_u_list.append(a_u_dag_list[i] @ a_u_list[i])
        n_d_list.append(a_d_dag_list[i] @ a_d_list[i])
        n_part_list.append(n_u_list[i] + n_d_list[i])

    # Build total number operator
    n_total_op = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        n_total_op += n_part_list[iiter]

    return n_u_list, n_d_list, n_part_list, n_total_op

In [7]:
def build_hubbard_hamiltonian(
    num_sites, hopping_strength, onsite_interaction, chemical_potential, s
):
    a_u_list, a_d_list, a_u_dag_list, a_d_dag_list = build_fermionic_operators(
        num_sites
    )
    n_u_list, n_d_list, n_part_list, n_total_op = build_number_operators(
        a_u_list, a_d_list, a_u_dag_list, a_d_dag_list
    )
    # Build Hubbard Hamiltonian with open boundary conditions
    # Following Eqn. 11 of https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.cond-mat.de/events/correl16/manuscripts/scalettar.pdf&ved=2ahUKEwjvus-1h4yHAxXIJzQIHdXNO1UQFnoECBAQAw&usg=AOvVaw1RrwrDGqwV43PYpA8JlhDW
    t = hopping_strength
    U = onsite_interaction
    mu = chemical_potential
    hopping_term = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    interaction_term = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for j in range(1, 1 + s):
        for iiter in range(num_sites - j):
            hopping_term += -1 * t * (a_u_dag_list[iiter] @ a_u_list[iiter + j])
            hopping_term += -1 * t * (a_d_dag_list[iiter] @ a_d_list[iiter + j])
            hopping_term += -1 * t * (a_u_dag_list[iiter + j] @ a_u_list[iiter])
            hopping_term += -1 * t * (a_d_dag_list[iiter + j] @ a_d_list[iiter])
        # print(iiter)
        
    # Onsite interaction term
    for iiter in range(num_sites):
        interaction_term += U * n_u_list[iiter] @ n_d_list[iiter]
        
    interaction_term = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        interaction_term += U * (
                (n_u_list[iiter] - 0.5 * sps.eye(4**num_sites))
                @ (n_d_list[iiter] - 0.5 * sps.eye(4**num_sites))
            )
    # Chemical potential, adjusted for half-filling
    chemical_potential_term = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        for jjter in range(num_sites):
            chemical_potential_term += -1 * (mu) * n_u_list[iiter] @ n_d_list[jjter]

    hami_one_dim_hubbard = (
        hopping_term + interaction_term + chemical_potential_term  # + constant_shift
    )

    np.set_printoptions(threshold=np.inf, linewidth=np.inf)
    # print(f"hopping_term = \n{hopping_term.toarray()}")
    # print(f"interaction_term = \n{interaction_term.toarray()}")
    # print(f"hami_one_dim_hubbard = \n{hami_one_dim_hubbard.toarray()}")
    # Check that the Hamiltonian is Hermitian
    assert (hami_one_dim_hubbard - hami_one_dim_hubbard.transpose().conj()).nnz == 0
    return (
        hami_one_dim_hubbard,
        a_u_list,
        a_d_list,
        a_u_dag_list,
        a_d_dag_list,
        n_u_list,
        n_d_list,
        n_part_list,
        n_total_op,
    )


def diagonalize_hubbard_hami(hami_one_dim_hubbard, num_states=2):
    eigvals, eigvecs = sps.linalg.eigsh(hami_one_dim_hubbard, k=num_states, which="SA")

    # eigvals += U/4 # Adjust for particle hole symmetric form of the Hamiltonian
    # Set near-zero values to zero
    eigvec = eigvecs[:, 0]
    eigvec[abs(eigvec) < 1e-10] = 0
    eigvec = eigvec / np.linalg.norm(eigvec)
    eigvec_sparse = sps.csr_array([eigvec])
    return eigvals, eigvec_sparse, eigvec


def get_observables(
    eigvals,
    eigvec_sparse,
    n_total_op,
    n_u_list,
    n_d_list,
    a_u_dag_list,
    a_d_list,
    a_d_dag_list,
    a_u_list,
    num_sites
):
    # print(f"Ground state energy from exact diagonalization = {eigvals[0]}")
    # print(f"Ground state from exact diagonalization = {eigvec_sparse}")
    # print(f"num non-zero elements = {eigvec_sparse.nnz}")
    measured_num_particles = (
        eigvec_sparse @ n_total_op @ eigvec_sparse.conj().transpose()
    )
    measured_num_particles = measured_num_particles[0, 0]
    # print(f"Total number of particles = {measured_num_particles}")
    # Get total s_z operator
    s_z = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        s_z += 0.5 * (n_u_list[iiter] - n_d_list[iiter])

    measured_s_z = eigvec_sparse @ s_z @ eigvec_sparse.conj().transpose()
    measured_s_z = measured_s_z[0, 0]
    # print(f"Total <Sz> = {measured_s_z}")

    # Get total s^2 operator
    s_plus = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    s_minus = sps.csr_matrix((4**num_sites, 4**num_sites), dtype=np.float_)
    for iiter in range(num_sites):
        s_plus += a_u_dag_list[iiter] @ a_d_list[iiter]
        s_minus += a_d_dag_list[iiter] @ a_u_list[iiter]

    s_squared = s_minus @ s_plus + s_z @ s_z + s_z
    measured_s_squared = eigvec_sparse @ s_squared @ eigvec_sparse.conj().transpose()
    measured_s_squared = measured_s_squared[0, 0]
    S_val = 0.5 * (-1 + np.sqrt(1 + 4 * measured_s_squared))
    # print(f"Total <S^2> = {measured_s_squared}")
    # print(f"Total S = {S_val}")

    # Get local moment
    m_squared_op = (2 * s_z) @ (2 * s_z)
    m_squared = eigvec_sparse @ m_squared_op @ eigvec_sparse.conj().transpose()

    m_squared = m_squared[0, 0]
    # print(f"Local moment squared <m^2> = {m_squared}")
    # print(f"Eigenvalues = {eigvals}")
    return measured_num_particles, measured_s_z, measured_s_squared, S_val, m_squared


In [10]:
s = 1
(
    hami_one_dim_hubbard,
    a_u_list,
    a_d_list,
    a_u_dag_list,
    a_d_dag_list,
    n_u_list,
    n_d_list,
    n_part_list,
    n_total_op,
) = build_hubbard_hamiltonian(
    
    4, 1, 1, 0, s
)

In [11]:
eigvals, eigvec_sparse, eigvec = diagonalize_hubbard_hami(hami_one_dim_hubbard, 12)
print(eigvals)

[-4.57536562 -3.90990407 -3.90990407 -3.90990407 -3.90990407 -3.59672212 -3.59672212 -3.59672212 -2.98733842 -2.98733842 -2.98733842 -2.96393603]


In [133]:
print(len(eigvec[eigvec != 0]))

36


In [126]:
print(eigvec)

[ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.         -0.00038114  0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.

In [127]:
measured_num_particles, measured_s_z, measured_s_squared, S_val, m_squared = (
    get_observables(
        eigvals,
        eigvec_sparse,
        n_total_op,
        n_u_list,
        n_d_list,
        a_u_dag_list,
        a_d_list,
        a_d_dag_list,
        a_u_list,
        4
    )
)
print(measured_num_particles)
print(measured_s_z)

5.999999999999998
0.0
